In [3]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k, recall_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 
from hnmchallenge.feature_manager import FeatureManager

In [2]:
dataset = StratifiedDataset()
dr = DataReader()

In [4]:
fm = FeatureManager(dataset, "train")

In [5]:
features_df = fm.create_features_df("cosine_recs_100_tw_True.feather")

Creating features df for training...
Loading item features...
join item features...
Loading user features...
join user features...
Final number of features loaded: 101


In [7]:
features_df.head()

,customer_id,article_id,itemknn_score,itemknn_rank,relevance,colour_group_code,department_no,Accessories_x,Blouses,Dressed,...,age,0_x,ACTIVE,LEFT CLUB,PRE-CREATE,0_y,Monthly,NONE,Regularly,FN
0,1,3161,0.010078,1,1,9,1676,0,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0
1,1,7534,0.009785,2,0,51,1338,0,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0
2,1,2997,0.007242,3,0,10,1676,0,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0
3,1,3503,0.007069,4,0,9,3948,1,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0
4,1,1482,0.006179,5,0,71,1747,0,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0


In [12]:
import math
TRAIN_PERC = 0.8
VAL_PERC = 0.1
TEST_PERC = 0.1
unique_users = features_df[DEFAULT_USER_COL].unique()
train_len = math.ceil(len(unique_users)*TRAIN_PERC)
val_len =math.ceil(len(unique_users)*VAL_PERC)
test_len =math.ceil(len(unique_users)*TEST_PERC)

np.random.seed(RANDOM_SEED)
np.random.shuffle(unique_users)
train_users, val_users, test_users = unique_users[:train_len], unique_users[train_len:train_len+val_len], unique_users[train_len+val_len:]

In [17]:
train_df = features_df[features_df[DEFAULT_USER_COL].isin(train_users)]
val_df = features_df[features_df[DEFAULT_USER_COL].isin(val_users)]
test_df = features_df[features_df[DEFAULT_USER_COL].isin(test_users)]

In [18]:
import xgboost as xgb

/raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [37]:
model = xgb.XGBRanker(  
    tree_method='gpu_hist',
    booster='gbtree',
    objective='rank:map',
    random_state=RANDOM_SEED, 
    learning_rate=1e-3,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=100, 
    subsample=0.75,
    n_gpu = -1
    )

In [29]:
X_train = train_df.loc[:, ~train_df.columns.isin([DEFAULT_USER_COL, DEFAULT_ITEM_COL, "relevance"])]
Y_train = train_df["relevance"].copy().values
qid_train = train_df[DEFAULT_USER_COL].copy().values

X_val = val_df.loc[:, ~val_df.columns.isin([DEFAULT_USER_COL, DEFAULT_ITEM_COL, "relevance"])]
Y_val = val_df["relevance"].copy().values
qid_val = val_df[DEFAULT_USER_COL].copy().values

X_test = test_df.loc[:, ~test_df.columns.isin([DEFAULT_USER_COL, DEFAULT_ITEM_COL, "relevance"])]
Y_test = test_df["relevance"].copy().values
qid_test = test_df[DEFAULT_USER_COL].copy().values

In [38]:
model.fit(X_train, Y_train, qid=qid_train, eval_set=[(X_val, Y_val)], eval_qid=[qid_val], eval_metric=["map@12"],
          verbose=True)

/raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[16:32:55] WARNING: ../src/learner.cc:576: 
Parameters: { "n_gpu" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBoostError: [16:33:12] ../src/tree/updater_gpu_hist.cu:770: Exception in gpu_hist: [16:33:12] ../src/c_api/../data/../common/device_helpers.cuh:428: Memory allocation error on worker 0: [16:33:12] ../src/c_api/../data/../common/common.h:45: ../src/common/device_helpers.cuh: 447: cudaErrorMemoryAllocation: out of memory
Stack trace:
  [bt] (0) /raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x314959) [0x7f6a12108959]
  [bt] (1) /raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x318203) [0x7f6a1210c203]
  [bt] (2) /raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x34d24e) [0x7f6a1214124e]
  [bt] (3) /raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x361ead) [0x7f6a12155ead]
  [bt] (4) /raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/xgb

In [10]:
final_features_df

,itemknn_score,itemknn_rank,colour_group_code,department_no,Accessories_x,Blouses,Dressed,Dresses Ladies,Dresses/Skirts girls,Jersey Basic,...,age,0_x,ACTIVE,LEFT CLUB,PRE-CREATE,0_y,Monthly,NONE,Regularly,FN
0,0.010078,1,9,1676,0,0,0,0,0,1,...,26.0,0,1,0,0,0,0,1,0,0.0
1,0.009785,2,51,1338,0,0,0,0,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0
2,0.007242,3,10,1676,0,0,0,0,0,1,...,26.0,0,1,0,0,0,0,1,0,0.0
3,0.007069,4,9,3948,1,0,0,0,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0
4,0.006179,5,71,1747,0,0,0,0,0,0,...,26.0,0,1,0,0,0,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48331695,0.011835,96,51,1422,0,0,0,0,0,0,...,22.0,0,1,0,0,0,0,1,0,0.0
48331696,0.011771,97,11,1322,0,0,0,1,0,0,...,22.0,0,1,0,0,0,0,1,0,0.0
48331697,0.011720,98,33,5999,0,0,0,0,0,0,...,22.0,0,1,0,0,0,0,1,0,0.0
48331698,0.011698,99,10,1647,0,0,0,0,0,0,...,22.0,0,1,0,0,0,0,1,0,0.0


In [ ]:
gss = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 7).split(final_features_df.drop(['customer_id','article_id','relevance']), groups=final_features_df['customer_id'])